# Computer Vision - Project
imports

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import h5py
import requests
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import tensorflow as tf
import h5py, requests, os
import matplotlib.patches as patches

In [ ]:
# IMG_SIZE = (64, 64)
# IMG_SHAPE = IMG_SIZE + (3,)
# base_model = tf.keras.applications.resnet(input_shape=IMG_SHAPE,
#                                                include_top=False,
#                                                weights='imagenet')
# base_model.trainable = False

Constants

In [ ]:
#ZIP_PATH = "https://drive.google.com/drive/folders/1jzHYpTwywUYA53nMGHVROSuVO14hEueq?usp=sharing/"
FILE_NAME ="SynthText_train.h5"
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
tf.keras.backend.clear_session()
SIZE = 128


In [ ]:
def sort_points(points, center):
    # calculate the angle of each point from the center point
    angles = np.arctan2(points[:, 1] - center[1], points[:, 0] - center[0])
    # sort the points by angle
    sorted_points = points[np.argsort(angles)]
    return sorted_points

In [ ]:
def draw_points(image, points, color=(255, 0, 0), radius=3):
    # create a copy of the image
    img = image.copy()
    # iterate over the points and draw them on the image
    for point in points:
        cv2.circle(img, tuple(map(int, point)), radius, color, -1)
    return img


In [ ]:
def get_bb(img, bbs, indx):
    x1 = int(bbs[0,0,indx])
    y1 = int(bbs[1,0,indx])
    x2 = int(bbs[0,1,indx])
    y2 = int(bbs[1,1,indx])
    x3 = int(bbs[0,2,indx])
    y3 = int(bbs[1,2,indx])
    x4 = int(bbs[0,3,indx])
    y4 = int(bbs[1,3,indx])
    # calculate bounding rectangle
    top_left_x = max(0, min([x1,x2,x3,x4]))
    top_left_y = max(0, min([y1,y2,y3,y4]))
    bot_right_x = max(0, max([x1,x2,x3,x4]))
    bot_right_y = max(0, max([y1,y2,y3,y4]))
    points = np.float32([[x1, y1], [x2, y2], [x3, y3], [x4, y4]])
    
    # create an empty image with the same shape as the input image
    mask = np.zeros_like(img[:,:,0])
    # create a list of the bounding box points in the correct format
    bounding_box = np.array([points], dtype=np.int32)
    # fill the area inside the bounding box with white
    cv2.fillPoly(mask, bounding_box, 1)
    # apply the mask to the image
    ####res = cv2.bitwise_and(img, mask)
    # try1 = np.array(np.where(mask == 255, 255, 0), dtype=np.uint8)
    # print(x)
    # inv_mask = cv2.bitwise_not(mask)
    # res2 = cv2.bitwise_and(img, inv_mask)
    # bb2 = np.int32([[top_left_x, bot_right_y], [bot_right_x, bot_right_y],[bot_right_x,top_left_y], [top_left_x,top_left_y]])
    # print(bb2)
    # mask2 = np.zeros_like(img)
    # # bb2 = np.array(frame, dtype=np.int32)
    # cv2.fillPoly(mask2, bb2, (255, 255, 255))
    mask = mask[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]
    res = img[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]

    #res = img[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]
    #flipping
    """"if(x2 < x1):
        res = cv2.flip(res, 1)
    if(y2 < y1):
        res = cv2.flip(res, 0)
    plt.imshow(mask)
    plt.show()
    print(mask.shape)"""
    return res, mask

In [ ]:
def font_to_num(font):
    if font == b'Alex Brush':
        return 0
    elif font == b'Titillium Web':
        return 1
    elif font == b'Sansation':
        return 2
    elif font == b'Open Sans':
        return 3
    else:
        return 4

In [ ]:
def prepare_img(img, bbs, index, size):
    cropped, mask = get_bb(img, bbs, index)
    # cropped = cv2.GaussianBlur(np.array(cropped), (3, 3), 0)
    # plt.imshow(cropped)
    # plt.show()
    # cropped = tf.image.convert_image_dtype(cropped, tf.float32)
    
    # cropped = tf.cast(cropped, tf.int32)
    # plt.imshow(cropped)
    # plt.show()
    # gray_mask = tf.image.rgb_to_grayscale(mask)
    # print(mask.shape)
    # print(cropped.shape)
    cropped = tf.image.rgb_to_grayscale(cropped)
    res = tf.cast(cropped, tf.float32) /255.0
    # mask = tf.cast(mask, tf.float32)
    # print(cropped.shape)
    # cropped = tf.expand_dims(cropped, 0)
    # mask = tf.expand_dims(mask, 2)
    # mask = tf.image.resize_with_pad(mask, size, size)
    # print(cropped.shape)
    # print(mask.shape)
    # cropped = tf.image.sobel_edges(cropped)
    # cropped = cropped**2
    # # print(cropped.shape)
    # cropped = tf.math.reduce_sum(cropped,axis=-1) # sum all magnitude components
    # cropped = tf.sqrt(cropped) # this is the image tensor you want
    # # print(cropped[0, :, :, :].shape)
    # # plt.imshow(cropped[0, :, :, :], cmap='gray')
    # # plt.show()
    # cropped = cropped[0, :, :, :]
    # plt.imshow(cropped, cmap ='gray')
    # plt.show()
    # plt.imshow(mask, cmap ='gray')
    # plt.show()
    # print(cropped.shape)
    # print(gray_mask.shape)
#    cropped = tf.cast(cropped, tf.int32)
    # mask = tf.cast(mask, tf.float32)
    # print(gray_mask)
    # res = tf.multiply(cropped, mask)
    
    # plt.imshow(mask, cmap ='gray')
    # plt.show()
    res = tf.image.resize_with_pad(res, size, size)
    # plt.imshow(res, cmap ='gray')
    # plt.show()
    return res#res

In [ ]:
def get_data_set(size: int):
    db = h5py.File(FILE_NAME, 'r')
    im_names = list(db['data'].keys())
    train_x = []
    train_y = []
    for i in range(0, len(im_names)-1):
        im = im_names[i]
        img  = db['data'][im][:]
        fonts = db['data'][im].attrs['font']
        txts = db['data'][im].attrs['txt']
        charBBs = db['data'][im].attrs['charBB']
        wordBBs = db['data'][im].attrs['wordBB']
        font_indx = 0 
        char_indx = 0
        # print(im)
        for j in range(0, len(txts)):
            cropped = prepare_img(img, wordBBs, j, size)
            train_x.append(cropped)
            train_y.append(font_to_num(fonts[font_indx]))
            font_indx += len(txts[j])
            #plt.imshow(cropped)
            #plt.show()
            for k in range(0, len(txts[j])):
                cropped = prepare_img(img, charBBs, char_indx, size)
                train_x.append(cropped)
                train_y.append(font_to_num(fonts[char_indx]))
                char_indx+=1
                #plt.imshow(cropped)
                #plt.show()
    return train_x, train_y

In [ ]:
train_x, train_y = get_data_set(SIZE)

In [ ]:
train_X = np.asarray(train_x, dtype="float") / 255.0
plt.imshow(train_x[3], cmap='gray')
print(train_X.shape)

In [ ]:
#cut the set for train and validation
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [ ]:
Y_train = np_utils.to_categorical(y_train, 5)
Y_test = np_utils.to_categorical(y_test, 5)

In [ ]:
# model = Sequential([
#   tf.keras.layers.RandomFlip("horizontal_and_vertical"),
#   tf.keras.layers.RandomRotation(0.2),
# ])
# print(x_train.shape)
# SIZE=128
from keras.layers import RandomFlip, RandomRotation, Masking
model = Sequential()

model.add(RandomFlip("horizontal_and_vertical"))
# model.add(RandomRotation(0.1))
model.add(Convolution2D(32, (5, 5), strides=(2,2), activation='relu', padding='same', input_shape=(SIZE,SIZE,1)))
model.add(Convolution2D(32, (5, 5), strides=(2,2), activation='relu', padding='same'))
# model.add(Masking(mask_value=0, input_shape=(SIZE, SIZE)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

In [ ]:

model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
X_train = np.array(x_train)
Y_train = np.array(Y_train)
# print(X_train.shape)
# print(Y_train.shape)
# plt.imshow(X_train[2], cmap='gray')


In [ ]:
# X_train = np.array(x_train, dtype='uint8')
print(X_train.shape)
print(Y_train.shape)

In [ ]:
model.fit(X_train, Y_train, batch_size=10, epochs=20, verbose=1)

In [ ]:
X_test = np.array(x_test)
#np.random.shuffle(Y_test)
score = model.evaluate(X_test, Y_test, verbose=0)

In [ ]:
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

In [ ]:
# res = model.predict(X_train[:7])
# print(res) 0.3924284279346466

In [ ]:
# print(Y_train[:7])

In [ ]:
#del model 
#tf.keras.backend.clear_session()